### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [1]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [2]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(x) for x in result / count]

In [3]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print(get_tuple_stream_mean(example(medium_stream()), 2))

['127.48', '137.48']
CPU times: user 3min 26s, sys: 212 ms, total: 3min 26s
Wall time: 3min 27s


# Решение

In [112]:
from collections import deque
from itertools import islice, tee
def movmean(stream, N):
    assert N == int(N) and N > 1, "N must be an integer > 1"
    streams = tee(stream,2)
    sum1 = 0
    sum2 = 0
    queue = deque(((sum1, sum2),))
    for i, val in enumerate(stream):
        sum1 += val
        sum2 += val**2
        queue.append((sum1, sum2))
        if i < N-1:
            continue
        s1, s2 = queue.popleft()
        mean = (sum1 - s1)/N
        variance = (sum2 - s2)/N - mean**2  # смещенная оценка
        yield mean, variance

In [113]:
def easy_minmaxmed(stream, N):
    """Т.к. easystream упорядоченный, то не нужно сортировать массив"""
    assert N == int(N) and N > 0, "N must be an integer >0"
    half = N//2
    queue=deque(islice(stream, N-1))
    for val in stream:
        queue.append(val)
        max_ = val
        med_ = queue[half]
        min_ = queue.popleft()
        yield min_, med_, max_

In [114]:
import bisect
def medium_minmaxmed(stream, N):
    assert N == int(N) and N > 0, "N must be an integer >0"
    half = N//2
    streams = tee(stream,2)
    queue = deque(islice(streams[0], N-1))
    sorted_list = sorted(list(islice(streams[1], N-1)))
    for val in stream:
        # Вставляем элемент в очередь и в отсортированный список
        bisect.insort_right(sorted_list, val)
        queue.append(val)
        # Максимум в правом конце отсортированного списка
        max_ = sorted_list[-1]
        med_ = sorted_list[half]
        # Минимум в левом конце
        min_ = sorted_list[0]
        sorted_list.remove(queue.popleft())
        yield min_, med_, max_

In [116]:
%%time
print("Mean and variance for different streams\n")
easy_tuple = get_tuple_stream_mean(movmean(easy_stream(), 1000), 2)
print("Easy: ", easy_tuple)
medium_tuple = get_tuple_stream_mean(movmean(medium_stream(), 1000), 2)
print("Medium: ", medium_tuple)
nightmare_tuple = get_tuple_stream_mean(movmean(nightmare_stream(), 1000), 2)
print("Nightmare: ", nightmare_tuple)

Mean and variance for different streams

Easy:  ['4999675.28', '83439.34']
Medium:  ['127.48', '5455.17']
Nightmare:  ['499880345.88', '83228908564027744.00']
CPU times: user 12min 34s, sys: 2.29 s, total: 12min 37s
Wall time: 12min 43s


In [117]:
%%time
easy_mmm = get_tuple_stream_mean(easy_minmaxmed(easy_stream(), 1000), 3)
print("easy_stream min med max", easy_mmm)

easy_stream min med max ['4999175.79', '4999675.78', '5000174.76']
CPU times: user 3min 41s, sys: 664 ms, total: 3min 41s
Wall time: 3min 43s


In [118]:
%%time
medium_mmm = get_tuple_stream_mean(medium_minmaxmed(medium_stream(), 1000), 3)
print("medium_stream min med max", medium_mmm)

medium_stream min med max ['0.02', '127.60', '254.98']
CPU times: user 5min 34s, sys: 372 ms, total: 5min 35s
Wall time: 5min 35s


In [119]:
%%time
nightmare_mmm = get_tuple_stream_mean(medium_minmaxmed(nightmare_stream(), 1000), 3)
print("nightmare_stream min med max", nightmare_mmm)

nightmare_stream min med max ['1017512.29', '500438415.64', '999017359.97']
CPU times: user 5min 41s, sys: 364 ms, total: 5min 41s
Wall time: 5min 42s
